In [130]:
import pandas as pd
import os
import csv
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

In [131]:
os.chdir('/Users/andiedonovan/myProjects/Youtube_Python_Project/AndiesBranch/data/') # change directory
train_data = pd.read_csv('OKGOcomments.csv', delimiter=";", skiprows=2, encoding='latin-1', engine='python') # read in the data
df = pd.read_csv('data.csv', delimiter="@@@", skiprows=2, encoding='utf-8', engine='python') # read in the user's data

In [132]:
df.columns = ["comment", "label"]

In [133]:
df.head(3)

,comment,label
0,The Biggest rape in the football history,NaN
1,I was at a camp during this time and didn't ha...,NaN
2,One of the best football games played ever.......,NaN


In [134]:
#df["comment"] = df["comment"].str.encode('ascii')

In [135]:
#df['comment'].iloc[0].encode('ascii', 'ignore')

In [136]:
train_data.columns = [
  'label',
  'comment','a','b'
]
train_data = train_data.drop(['a', 'b'], 1).dropna()
for row in range(len(train_data)):
    line = train_data.iloc[row,1]
    train_data.iloc[row,1] = re.sub("[^a-zA-Z]", " ", line)

In [137]:
train_data.head(5)
#data = data.ix[:, 0]

,label,comment
0,-1.0,Everyone knows brand s papers from But No on...
1,0.0,Your paper cut balance is
2,1.0,OH SHIT WHEN I SAW THIS ON MY FRONT PAGE ...
3,1.0,Blowing my mind yet again
4,0.0,Should have gone with Dunder Mifflin


In [138]:
'''
for row in range(len(df)):
    line = df.loc[row, "comment"]
    #line = data.iloc[row,0]
    df.loc[row,"comment"] = re.sub("[^a-zA-Z]", " ", line)
'''

'\nfor row in range(len(df)):\n    line = df.loc[row, "comment"]\n    #line = data.iloc[row,0]\n    df.loc[row,"comment"] = re.sub("[^a-zA-Z]", " ", line)\n'

In [139]:
df.head(5)
df2 = df

In [140]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
import sklearn # machine learning
from sklearn.model_selection import train_test_split # splitting up data
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andiedonovan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [141]:
sw = stopwords.words('english')
ps = PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
tfidf = TfidfVectorizer()

In [142]:
df["comment"]= df["comment"].astype(str) 

In [143]:
#df["comment"] = df["comment"].applymap(str)
for row in range(len(df)):
        line = df.loc[row, "comment"]
        #line = data.iloc[row,0]
        df.loc[row,"comment"] = re.sub("[^a-zA-Z]", " ", line)

In [144]:
labels = train_data['label']

In [145]:
def nlpFunction(a):
    a['com_token']=a['comment'].str.lower().str.split()
    a['com_remv']=a['com_token'].apply(lambda x: [y for y in x if y not in sw])
    a["com_lemma"] = a['com_remv'] \
        .apply(lambda x : [lemmatizer.lemmatize(y) for y in x]) # lemmatization
    a['com_stem']=a['com_lemma'] \
        .apply(lambda x : [ps.stem(y) for y in x]) # stemming
    return df

In [146]:
df = nlpFunction(df)
train_data = nlpFunction(train_data)

In [147]:
train_data.head(3)

,comment,label,com_token,com_remv,com_lemma,com_stem
0,The Biggest rape in the football history,NaN,"[the, biggest, rape, in, the, football, history]","[biggest, rape, football, history]","[biggest, rape, football, history]","[biggest, rape, footbal, histori]"
1,I was at a camp during this time and didn t ha...,NaN,"[i, was, at, a, camp, during, this, time, and,...","[camp, time, access, internet, tv, said, germa...","[camp, time, access, internet, tv, said, germa...","[camp, time, access, internet, tv, said, germa..."
2,One of the best football games played ever ...,NaN,"[one, of, the, best, football, games, played, ...","[one, best, football, games, played, ever, ama...","[one, best, football, game, played, ever, amaz...","[one, best, footbal, game, play, ever, amaz, m..."


In [148]:
train_data['label'] = labels

In [149]:
train_data.head(3)

,comment,label,com_token,com_remv,com_lemma,com_stem
0,The Biggest rape in the football history,-1.0,"[the, biggest, rape, in, the, football, history]","[biggest, rape, football, history]","[biggest, rape, football, history]","[biggest, rape, footbal, histori]"
1,I was at a camp during this time and didn t ha...,0.0,"[i, was, at, a, camp, during, this, time, and,...","[camp, time, access, internet, tv, said, germa...","[camp, time, access, internet, tv, said, germa...","[camp, time, access, internet, tv, said, germa..."
2,One of the best football games played ever ...,1.0,"[one, of, the, best, football, games, played, ...","[one, best, football, games, played, ever, ama...","[one, best, football, game, played, ever, amaz...","[one, best, footbal, game, play, ever, amaz, m..."


In [150]:
df["com_stem_str"] = df["com_stem"].apply(', '.join)
train_data["com_stem_str"] = train_data["com_stem"].apply(', '.join)

X_train, X_test, Y_train, Y_test = train_test_split(
                                    train_data["com_stem_str"], train_data["label"], 
                                    test_size=0.25, 
                                    random_state=42)

In [151]:
tfidf = TfidfVectorizer()
xtrain = tfidf.fit_transform(X_train) # transform and fit training data
xtest = tfidf.transform(X_test) # transform test data from fitted transformer

data_trans= tfidf.transform(train_data["com_stem_str"]) # transform entire dataset for cross validation

In [152]:
df_trans = tfidf.transform(df["com_stem_str"])

In [153]:
df_trans

<259x553 sparse matrix of type '<class 'numpy.float64'>'
	with 1317 stored elements in Compressed Sparse Row format>

In [154]:
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn import svm # support vector machine
from sklearn import metrics # for accuracy/ precision
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent

In [155]:
lr = LogisticRegression(solver='sag', max_iter=100, random_state=42, multi_class="multinomial") # set multinomial setting for multiclass data

In [156]:
lr.fit(xtrain, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [157]:
xtest

<65x553 sparse matrix of type '<class 'numpy.float64'>'
	with 294 stored elements in Compressed Sparse Row format>

In [158]:
df_trans

<259x553 sparse matrix of type '<class 'numpy.float64'>'
	with 1317 stored elements in Compressed Sparse Row format>

In [159]:
lr_predict = lr.predict(df_trans)
#lr_acc = metrics.accuracy_score(Y_test, lr_predict)
#print('We obtained ', round(lr_acc, 6), '% accuracy for the logistic regression model')

In [160]:
#lr_predict

In [161]:
#dictionary = dict(zip(lr_predict, df2['comment']))

In [162]:
Table = pd.DataFrame(columns=['label','comment'])

In [163]:
Table['label'] = lr_predict
Table['comment'] = df2['comment']

In [164]:
Table

,label,comment
0,0.0,The Biggest rape in the football history
1,0.0,I was at a camp during this time and didn t ha...
2,1.0,One of the best football games played ever ...
3,1.0,Haha Never Gets Old
4,0.0,The most legend Brazil defenders and players o...
5,1.0,Well deserved brazil well deserved This matc...
6,1.0,My brother was watching the game in the living...
7,1.0,LMFAOOOOOOOO
8,1.0,I like germany team
9,1.0,whatever germans learnt it from South American...


In [165]:
type(Table['label'])
#Table['label'].astype(str)

pandas.core.series.Series

In [166]:
Table.label.apply(str)

0       0.0
1       0.0
2       1.0
3       1.0
4       0.0
5       1.0
6       1.0
7       1.0
8       1.0
9       1.0
10      1.0
11      1.0
12      1.0
13      1.0
14      1.0
15      1.0
16      0.0
17      1.0
18     -1.0
19      1.0
20      0.0
21      1.0
22      0.0
23      1.0
24      1.0
25      1.0
26      1.0
27     -1.0
28      1.0
29      1.0
       ... 
229     1.0
230     1.0
231     1.0
232     0.0
233     0.0
234     1.0
235     0.0
236     0.0
237     1.0
238     1.0
239     0.0
240     1.0
241     1.0
242     1.0
243     1.0
244     1.0
245     1.0
246     1.0
247     1.0
248     1.0
249     1.0
250     1.0
251     0.0
252     1.0
253    -1.0
254     1.0
255     0.0
256     1.0
257     0.0
258     1.0
Name: label, Length: 259, dtype: object

In [167]:
Table

,label,comment
0,0.0,The Biggest rape in the football history
1,0.0,I was at a camp during this time and didn t ha...
2,1.0,One of the best football games played ever ...
3,1.0,Haha Never Gets Old
4,0.0,The most legend Brazil defenders and players o...
5,1.0,Well deserved brazil well deserved This matc...
6,1.0,My brother was watching the game in the living...
7,1.0,LMFAOOOOOOOO
8,1.0,I like germany team
9,1.0,whatever germans learnt it from South American...


In [168]:
Table['label']= Table['label'].apply(str)
mapping = {'0.0':"Neutral", '1.0':"Positive", '-1.0':"Negative"}
Table = Table.replace({'label': mapping})

In [169]:
Table.groupby('label').count()

,comment
label,
Negative,15
Neutral,95
Positive,149


In [170]:
'''
sentiments = ["Positive", "Negative", "Neutral"]

for i in range(1,len(sentiments)):
    sent = Table[Table['label']==i]
    sent_len = len(Table[Table['label']==i])
    if sent_len>1:
        print("There were %d %s comments" % (sent_len, sent))
    elif sent_len==1:
        print("There was %d %s comment" % (sent_len, sent))
    else: 
        print("There were no %s comments" % (sent))      
'''        

'\nsentiments = ["Positive", "Negative", "Neutral"]\n\nfor i in range(1,len(sentiments)):\n    sent = Table[Table[\'label\']==i]\n    sent_len = len(Table[Table[\'label\']==i])\n    if sent_len>1:\n        print("There were %d %s comments" % (sent_len, sent))\n    elif sent_len==1:\n        print("There was %d %s comment" % (sent_len, sent))\n    else: \n        print("There were no %s comments" % (sent))      \n'

In [171]:
pos = len(Table[Table['label']=="Positive"])
neg = len(Table[Table['label']=="Negative"])
neu = len(Table[Table['label']=="Neutral"])

In [172]:
pos

149

In [173]:
#from __future__ import division
pos
total = pos + neg + neu

neg_ratio = round(neg / float(total), 2) * 100
pos_ratio = round(pos / float(total), 2) * 100
neu_ratio = round(neu / float(total), 2) * 100

In [174]:
print("The data was %d percent positive, %d percent negative, and %d percent neutral..." %(pos_ratio, neg_ratio, neu_ratio))

The data was 57 percent positive, 6 percent negative, and 37 percent neutral...


In [175]:
#print("The data was {} percent positive, {} percent negative, and {} percent neutral...".format(pos_ratio, neg_ratio, neu_ratio))

The data was 57.99999999999999 percent positive, 6.0 percent negative, and 37.0 percent neutral...
